In [19]:
import tensorflow as tf 
import os
from urllib import request 
from pathlib import Path
import numpy as np
import matplotlib.pyplot as pyplot

In [5]:
IMAGE_SIZE = (224, 224)

In [7]:
model = tf.keras.applications.VGG16(
    include_top=False, weights="imagenet", input_tensor=None, input_shape=None, classes=1000
)

58892288/58889256 [==============================] - 8s 0us/step


In [8]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [10]:
model.save("VGG16_full_model.h5")

In [13]:
model.layers[1].padding

'same'

In [14]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [20]:
# create a directory
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")

ROOT_DATA_DIR = 'data'
create_dirs(ROOT_DATA_DIR)

data_zip_file = "data.zip"
data_zip_path = os.path.join(ROOT_DATA_DIR, data_zip_file)

if not os.path.exists(data_zip_path):
    print("Downloading data")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename: {filename} created with info \n {headers}")

else:
    print(f"file is already present")

data directory created
filename: data\data.zip created with info 
 Content-Type: application/zip
Content-Length: 47286322
Connection: close
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
Date: Sat, 02 Apr 2022 01:13:43 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
X-Cache: Hit from cloudfront
Via: 1.1 5c22f10a32e5d064bbf36cb39af79a98.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: MAA51-P3
X-Amz-Cf-Id: gKu9HX8ca5KZDPQC62UvTLS0pnuxf0aBcHburdsNnDXxMfEh-B4AZw==
Age: 50396




In [24]:
from zipfile import ZipFile

unzip_data_dirname = "unzip_data_dir"
unzip_data_dir = os.path.join(ROOT_DATA_DIR, unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir, exist_ok=True)
    with ZipFile(data_zip_path, 'r') as zip:
        zip.extractall(unzip_data_dir)

else:
    print(f"data already extracted")

data already extracted


In [25]:
main_data_dir = Path("D:/DS Ineuron/Projects/CNN_VGG16/CNN_VGG16/notebook/data/unzip_data_dir/hymenoptera_data/train")

In [26]:
BATCH_SIZE = 32
pixels = 224
IMAGE_SIZE = (pixels, pixels)

In [29]:
datagen_kwargs = dict(rescale=1./255, validation_split=0.2)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

In [32]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

valid_generator = valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation = False

if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotational_range=20,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        **datagen_kwargs)
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    main_data_dir, subset = "training", shuffle=True, **dataflow_kwargs)


Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [36]:
RGB_SIZE = (pixels, pixels, 3)

vgg = tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_SIZE,
    weights="imagenet",
    include_top=False
)

In [37]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [38]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_4             : True
block1_conv1        : True
block1_conv2        : True
block1_pool         : True
block2_conv1        : True
block2_conv2        : True
block2_pool         : True
block3_conv1        : True
block3_conv2        : True
block3_conv3        : True
block3_pool         : True
block4_conv1        : True
block4_conv2        : True
block4_conv3        : True
block4_pool         : True
block5_conv1        : True
block5_conv2        : True
block5_conv3        : True
block5_pool         : True


In [39]:
for layer in vgg.layers:
    layer.trainable = False

In [40]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_4             : False
block1_conv1        : False
block1_conv2        : False
block1_pool         : False
block2_conv1        : False
block2_conv2        : False
block2_pool         : False
block3_conv1        : False
block3_conv2        : False
block3_conv3        : False
block3_pool         : False
block4_conv1        : False
block4_conv2        : False
block4_conv3        : False
block4_pool         : False
block5_conv1        : False
block5_conv2        : False
block5_conv3        : False
block5_pool         : False


In [41]:
vgg.output

<tf.Tensor 'block5_pool/MaxPool_3:0' shape=(None, 7, 7, 512) dtype=float32>

In [49]:
CLASSES = 2

x = tf.keras.layers.Flatten()(vgg.output)
prediction = tf.keras.layers.Dense(CLASSES, activation = 'relu')(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [54]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"]
)

In [55]:
EPHOS = 1

history = model.fit(
    train_generator, epochs=EPHOS, validation_data=valid_generator
)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.